In [55]:
## Loading relevant packages
library(slackr)
library(AirSensor)
library(MazamaSpatialUtils)
library(lubridate)
library(dplyr)

## Loading Spatial data 
## More details on Air Sensor Git hub page if it's not setup already
initializeMazamaSpatialUtils("/home/ubuntu/Git/Sensor_Alert/Spatial_data")

## Settng the Country Code API link and look back days 
pas <- pas_createNew(countryCodes = "IN",
                     baseUrl = 'https://www.purpleair.com/json?all=true',lookbackDays = 30,
                     includePWFSL = F)
## Changing the time zone to Asia/Kolkata
pas$lastSeenDate <- with_tz(pas$lastSeenDate,tz="Asia/Kolkata")



In [56]:
SAMOSA_pas = pas%>% pas_filter(!grepl("B",label),grepl("SAMOSA",label))
tail(SAMOSA_pas)


ID,label,DEVICE_LOCATIONTYPE,THINGSPEAK_PRIMARY_ID,THINGSPEAK_PRIMARY_ID_READ_KEY,THINGSPEAK_SECONDARY_ID,THINGSPEAK_SECONDARY_ID_READ_KEY,latitude,longitude,pm25,⋯,deviceID,locationID,deviceDeploymentID,airDistrict,pwfsl_closestDistance,pwfsl_closestMonitorID,sensorManufacturer,targetPollutant,technologyType,communityRegion
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
97689,SAMOSA_0049,outside,1287287,SZQ6RG28WORNOL7M,1287288,1MH4T3WJCANV7PLM,28.54758,77.19093,21.72,⋯,97689,3c04d0c7632e85f5,3c04d0c7632e85f5_97689,NA,NA,NA,Purple Air,PM,consumer-grade,NA
97907,SAMOSA_0050,outside,1287790,R3KAPF9GP9XEVA5D,1287791,I9A3FTKH1DVHTTS8,28.54748,77.19141,23.95,⋯,97907,6c9bc0c5f03d8e83,6c9bc0c5f03d8e83_97907,NA,NA,NA,Purple Air,PM,consumer-grade,NA
97697,SAMOSA_0051,outside,1287303,BK5LRXVU9LNWTDGS,1287304,3ZZW3Q9YMFN5TBDT,28.54029,77.18483,26.02,⋯,97697,103372679e42fe77,103372679e42fe77_97697,NA,NA,NA,Purple Air,PM,consumer-grade,NA
97753,SAMOSA_0052,outside,1287437,988HE74NTKFBN8MC,1287438,17KRJDCG7LYLLAHK,28.54744,77.19199,29.28,⋯,97753,8c9420d1c423e4bd,8c9420d1c423e4bd_97753,NA,NA,NA,Purple Air,PM,consumer-grade,NA
97989,SAMOSA_0054,outside,1287990,N6PVESJWBK31CCEN,1287991,XE9JB4FEIGLO1IIN,28.54749,77.19135,24.28,⋯,97989,f3a2c9631da30e7d,f3a2c9631da30e7d_97989,NA,NA,NA,Purple Air,PM,consumer-grade,NA
97911,SAMOSA_0055,outside,1287801,9K58CZTOE1CQGLEC,1287802,N9LLDK29AYM9FB49,28.54673,77.18855,25.76,⋯,97911,2d9016624b6c451c,2d9016624b6c451c_97911,NA,NA,NA,Purple Air,PM,consumer-grade,NA


In [58]:
#Send notification to slack channel
time <- Sys.time()
## Changing the time zone to UTC
time <- with_tz(time,tz="Asia/Kolkata")
## Creating one hour lag for the test (If any sensor is inactive for more than an hour we send an alert)
time_check = time-minutes(5)
## Finding any inactive sensors
Inactive_sensors = SAMOSA_pas %>% filter(time_check>lastSeenDate) %>% select(label,lastSeenDate)

dim(Inactive_sensors)[1]

[1] 49

In [59]:
## Adding the R environment file. This file contains the API used for analysis
readRenviron("/home/ubuntu/Git/API.Renviron")

slackr_setup(channel = "pa_sensor_alert",
             bot_user_oauth_token =  Sys.getenv('bot_user_oauth_token'),
             incoming_webhook_url = Sys.getenv('incoming_webhook_url'))

[1] "Successfully connected to Slack"

In [45]:
if (dim(Inactive_sensors)[1] > 0){
    
    slackr("Time of Query",time)
    ## Sending the Slack alert if the sensors is inactive at any moment
    slackr("Name of Inactive sensors at the moment", pander::pandoc.table(nrow(Inactive_sensors)))
    slackr("Name of Inactive sensors at the moment", pander::pandoc.table(Inactive_sensors))
}